In [167]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from collections import Counter
import math

- [x] Capturar Número de Vagas
- [x] Criar lista de links
- [x] Criar lista com descrições vagas
- [x] transformar em um DataFrame
- [x] Calcular tf_idf de palavras por vaga
- [ ] Recomendar 10 melhores vagas a partir de palavras-chave inseridas pelo usuario

# Get number of jobs

In [1]:
from bs4 import BeautifulSoup
import requests


In [14]:
paginator = 0
r = requests.get(f'https://br.indeed.com/empregos?q=cientista+de+dados&l=brasil&start={paginator}')
page = BeautifulSoup(r.content, 'html.parser')

In [62]:
n_jobs = int(pagina.find(id='searchCountPages').text.split()[3])
n_jobs

246

# Get opportunity links

In [63]:
job_links = []
for i in range(0, n_jobs, 10):
    r = requests.get(f'https://br.indeed.com/empregos?q=cientista+de+dados&l=brasil&start={i}')
    page = BeautifulSoup(r.content, 'html.parser')
    anchors = page.find_all('a', class_='jobtitle')
    for a in anchors:
        job_links.append('https://www.indeed.com' + a.get('href'))

# Get list of job description

In [64]:
job_description = []
bad_attemps = []
for link in job_links:
    r = requests.get(link)
    page = BeautifulSoup(r.content, 'html.parser')
    try:
        description = page.find('div', class_='jobsearch-jobDescriptionText').text.replace('\n', ' ')
    except:
        bad_attemps.append(link)
        continue
    job_description.append(description)
        
for link in bad_attemps:
    job_links.remove(link)

In [65]:
print(len(job_links))
print(len(job_description))

372
372


In [66]:
jobs = pd.DataFrame({'URL': job_links, 'description': job_description})

In [201]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"

def remove_symbols(description):
    for i in symbols:
        description = description.replace(i, ' ')
    return description

def remove_singles(description):
    new_text = ""
    for word in description.split(' '):
        if len(word) > 1:
            new_text += " " + word
    return new_text

# transform to lowercase
jobs['description'] = jobs['description'].apply(lambda x: x.lower())

# removing the symbols
jobs['description'] = jobs['description'].apply(lambda x: remove_symbols(x))

# removing single characters
jobs['description'] = jobs['description'].apply(lambda x: remove_singles(x))
jobs

,URL,description,word_count,tf,tfidf
0,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=bb7e03808b04c...,ajudamos empresas nacionais internacionais in...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=781d6e38e647e...,atribuições projetar soluções para armazenar ...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
...,...,...,...,...,...
367,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
368,https://www.indeed.com/rc/clk?jk=e65cb176dca08...,como você pode transformar brasil por meio da...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
369,https://www.indeed.com/rc/clk?jk=3a815ac96c8ac...,como desenvolvedor backend ou fullstack profi...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
370,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."


# Tf_idf calc

### Set of words in corpus

In [202]:
words = []
for tf in jobs['description']:
    words.extend(tf.split(' '))
word_set = set(words)
word_set.remove('')

### word counting

In [203]:
def word_count(description):
    word_dict = {}
    for i in word_set:
        word_dict[i] = description.count(i)
    return word_dict

In [204]:
jobs['word_count'] = jobs['description'].apply(word_count)
jobs

,URL,description,word_count,tf,tfidf
0,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=bb7e03808b04c...,ajudamos empresas nacionais internacionais in...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=781d6e38e647e...,atribuições projetar soluções para armazenar ...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
...,...,...,...,...,...
367,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
368,https://www.indeed.com/rc/clk?jk=e65cb176dca08...,como você pode transformar brasil por meio da...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
369,https://www.indeed.com/rc/clk?jk=3a815ac96c8ac...,como desenvolvedor backend ou fullstack profi...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
370,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."


### Term frequencies in job description

In [205]:
def tf(counter):
    tf_dict = {}
    counter_len = sum(counter.values())
    for key, value in counter.items():
        tf_dict[key] = value / counter_len
    return tf_dict

In [206]:
jobs['tf'] = jobs['word_count'].apply(tf)
jobs

,URL,description,word_count,tf,tfidf
0,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=bb7e03808b04c...,ajudamos empresas nacionais internacionais in...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=781d6e38e647e...,atribuições projetar soluções para armazenar ...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
...,...,...,...,...,...
367,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
368,https://www.indeed.com/rc/clk?jk=e65cb176dca08...,como você pode transformar brasil por meio da...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
369,https://www.indeed.com/rc/clk?jk=3a815ac96c8ac...,como desenvolvedor backend ou fullstack profi...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
370,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."


### Inverse Document Frequencies

In [207]:
doc_freq = {}
for word in word_set:
    doc_freq[word] = 0

for word in word_set:
    for word_count in jobs['word_count']:
        if word_count[word] != 0:
            doc_freq[word] += 1

In [208]:
idf = {}
for word, freq in doc_freq.items():
    idf[word] = math.log(len(jobs)/(1 + freq))

### TF-IDF calc

In [209]:
def tfidf_calc(tf):
    tfidf = {}
    for word in tf.keys():
        tfidf[word] = tf[word] * idf[word]
    return tfidf

In [210]:
jobs['tfidf'] = jobs['tf'].apply(tfidf_calc)
jobs

,URL,description,word_count,tf,tfidf
0,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=bb7e03808b04c...,ajudamos empresas nacionais internacionais in...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=781d6e38e647e...,atribuições projetar soluções para armazenar ...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
...,...,...,...,...,...
367,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
368,https://www.indeed.com/rc/clk?jk=e65cb176dca08...,como você pode transformar brasil por meio da...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
369,https://www.indeed.com/rc/clk?jk=3a815ac96c8ac...,como desenvolvedor backend ou fullstack profi...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."
370,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'cadastre': 0, 'comitê': 0, 'who': 0, 'cumpri...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '...","{'cadastre': 0.0, 'comitê': 0.0, 'who': 0.0, '..."


# Most important words of all jobs

In [211]:
word_importance = {}

In [212]:
for word in word_set:
    word_importance[word] = 0

In [213]:
for tfidf in jobs['tfidf']:
    for word, importance in word_importance.items():
        word_importance[word] += tfidf[word]

In [214]:
df_importance = pd.DataFrame({'word': word_importance.keys(), 'importance': word_importance.values()})
df_importance.sort_values(['importance'], ascending=False).head(20)

,word,importance
4967,learning,0.366878
3071,learn,0.343518
1201,machine,0.329121
4043,vale,0.301044
4676,cs,0.295955
379,modelo,0.292624
1023,you,0.288180
6636,analytic,0.272475
3074,analytics,0.270232
7056,modelagem,0.264708
